In [1]:
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from torchvision import transforms, datasets
import torch.utils.data as data

torch.backends.cudnn.benchmark = True

import deeptriplet.models
import deeptriplet.datasets
import deeptriplet.transforms as ctr

In [2]:
model_path_random = "/scratch-second/yardima/logs/semseg/deeplabv3p/aug-spatial-early-multi/run_5/models/deeplabv3p-multi_epoch-59.pth"
d1 = torch.load(model_path_random)

from deeptriplet.models.deeplabv3p import DeepLabSpatialEarly
net = DeepLabSpatialEarly(backbone='resnet', num_classes=64, dynamic_coordinates=True, sync_bn=True, freeze_bn=False, pretrained=False)
net = net.cuda()
net.load_state_dict(d1['model_dict'])

net = net.eval().cuda();
embed_dim = 64

In [5]:
dataset = deeptriplet.datasets.Davis2017(root="/scratch-second/yardima/datasets/Davis2017/", 
                                         train=False,
                                         transforms=transforms.Compose([ctr.ToTensor(),
                                                                        ctr.NormalizeImagenet()]))

In [6]:
import os

In [9]:
if not os.path.isdir("/scratch-second/yardima/temp/davis_untrained0"):
    os.mkdir("/scratch-second/yardima/temp/davis_untrained0")

for seq in range(len(dataset)):
    n_frames = dataset.count_frames(seq)
    
    for i in range(0, n_frames, 2):
        with torch.no_grad():
            frame0 = dataset[seq, i]
            img = frame0['image'].cuda()[None,:,:,:]
            lbl = frame0['label'].numpy()[::2,::2].reshape(-1)

            embed = net.forward(img)
            #embed = F.upsample(embed, scale_factor=0.5, mode="bilinear")
            h = embed.shape[2]
            w = embed.shape[3]
            embed = embed.squeeze().cpu().numpy()
            embed = np.transpose(embed, axes=(1,2,0))[::2,::2,:].reshape(-1, embed_dim)
    
            np.save("/scratch-second/yardima/temp/davis_untrained0/e{}_{}.npz".format(seq,i), embed)    
            np.save("/scratch-second/yardima/temp/davis_untrained0/l{}_{}.npz".format(seq,i), lbl)    

In [1]:
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt

import deeptriplet.models
import deeptriplet.datasets
import deeptriplet.transforms as ctr
import os
from torchvision import transforms, datasets

from sklearn.neighbors import KNeighborsClassifier as KNN

In [2]:
def _fast_hist(label_true, label_pred, n_class):
    mask = (label_true >= 0) & (label_true < n_class)
    hist = np.bincount(
        n_class * label_true[mask].astype(int) + label_pred[mask],
        minlength=n_class ** 2,
    ).reshape(n_class, n_class)
    return hist

In [ ]:
%%time
n_seq = 30

miou = 0.0
mcontour = 0.0
list_j = []

for seq in range(n_seq):
    embed_init = np.load("/srv/glusterfs/yardima/temp/davis_untrained0/e{}_{}.npy".format(seq,0))
    lbl_init = np.load("/srv/glusterfs/yardima/temp/davis_untrained0/l{}_{}.npy".format(seq,0))
    
    knn = KNN(n_neighbors=3, n_jobs=32)
    knn.fit(embed_init, lbl_init)
    
    n_objects = lbl_init.max() + 1
    
    n_frames = 0
    
    i = 2
    hist = None
    while os.path.isfile("/srv/glusterfs/yardima/temp/davis_untrained0/e{}_{}.npy".format(seq,i)):
        
        embed = np.load("/srv/glusterfs/yardima/temp/davis_untrained0/e{}_{}.npy".format(seq,i)) 
        lbl = np.load("/srv/glusterfs/yardima/temp/davis_untrained0/l{}_{}.npy".format(seq,i))    

        yhat = knn.predict(embed)

        if hist is None:
            hist = _fast_hist(lbl, yhat, n_objects)
        else:
            hist += _fast_hist(lbl, yhat, n_objects)
        
        print(np.nanmean(np.diag(hist) / (hist.sum(axis=1) + hist.sum(axis=0) - np.diag(hist))))
        n_frames += 1
        print(i)
        i+=2
    
    iou = np.nanmean(np.diag(hist) / (hist.sum(axis=1) + hist.sum(axis=0) - np.diag(hist)))
    print(iou)
    list_j.append(iou)
    
    miou += iou
    
miou /= n_seq

0.9465679454673136
2
0.9335586060539797
4
0.9245435527714028
6
0.9089666978913025
8
0.9013440833434186
10
0.8910479088920636
12
0.8817463805214655
14
0.8688418227279652
16
0.8533351943792412
18
0.8321480405936338
20
0.819477348892792
22
0.8079871482372162
24
0.8000069602078052
26
0.7916528735791791
28
0.7839979357706186
30
0.777727770417748
32
0.7723082126131867
34
0.7672230537532219
36
0.7632207776813615
38
0.7607380848621371
40
0.7583736762971475
42
0.7554834967844499
44
0.7530232715372688
46
0.7518505931839302
48
0.7513674446303994
50
0.7511150925368036
52
0.7518105507839682
54
0.7523547612507916
56
0.7525700765381304
58
0.753178360451202
60
0.7542700623115642
62
0.7556835360246046
64
0.7563533066894185
66
0.7564038135465102
68
0.7564038135465102
0.9818466505679392
2
0.978274419613907
4
0.977704729037862
6
0.9771445748366685
8
0.9768550153323925
10
0.9763585019613514
12
0.975690165253246
14
0.9750813908187446
16
0.9739525715280573
18
0.9736928470904089
20
0.9735110901011496
22
0.973

In [ ]:
np.save("/srv/glusterfs/yardima/temp/davis_untrained0/miou", miou)
np.save("/srv/glusterfs/yardima/temp/davis_untrained0/list", list_j)

In [ ]:
%%time
n_seq = 30

miou = 0.0
mcontour = 0.0
list_j = []

for seq in range(n_seq):
    embed_init = np.load("/srv/glusterfs/yardima/temp/davis_untrained0/e{}_{}.npy".format(seq,0))
    lbl_init = np.load("/srv/glusterfs/yardima/temp/davis_untrained0/l{}_{}.npy".format(seq,0))
    
    knn = KNN(n_neighbors=1, n_jobs=32)
    knn.fit(embed_init, lbl_init)
    
    n_objects = lbl_init.max() + 1
    
    n_frames = 0
    
    i = 2
    hist = None
    while os.path.isfile("/srv/glusterfs/yardima/temp/davis_untrained0/e{}_{}.npy".format(seq,i)):
        
        embed = np.load("/srv/glusterfs/yardima/temp/davis_untrained0/e{}_{}.npy".format(seq,i)) 
        lbl = np.load("/srv/glusterfs/yardima/temp/davis_untrained0/l{}_{}.npy".format(seq,i))    

        yhat = knn.predict(embed)

        if hist is None:
            hist = _fast_hist(lbl, yhat, n_objects)
        else:
            hist += _fast_hist(lbl, yhat, n_objects)
        
        print(np.nanmean(np.diag(hist) / (hist.sum(axis=1) + hist.sum(axis=0) - np.diag(hist))))
        n_frames += 1
        print(i)
        i+=2
    
    iou = np.nanmean(np.diag(hist) / (hist.sum(axis=1) + hist.sum(axis=0) - np.diag(hist)))
    print(iou)
    list_j.append(iou)
    
    miou += iou
    
miou /= n_seq

In [ ]:
np.save("/srv/glusterfs/yardima/temp/davis_untrained0/miou2", miou)
np.save("/srv/glusterfs/yardima/temp/davis_untrained0/list2", list_j)